In [ ]:
import mysql.connector
import AWSIoTPythonSDK.MQTTLib as AWSIoTPyMQTT
import time
import random
from matplotlib import pyplot as plot
import sys
import json
import pyotp
import qrcode

endpoint = "a3pr6d9ch5nj20-ats.iot.us-east-1.amazonaws.com"
client_id = "iotunderdogs"
path_certificate = "./Certificados/certificadoiot.crt"
path_privatekey = "./Certificados/privateiot.key"
path_rootca1 = "./Certificados/AmazonRootCA1.pem"

my_awsmqtt_client = AWSIoTPyMQTT.AWSIoTMQTTClient(client_id)  
my_awsmqtt_client.configureEndpoint(endpoint, 8883) 
my_awsmqtt_client.configureCredentials(path_rootca1, path_privatekey, path_certificate)

key = "italaoPikudo"

totp = pyotp.TOTP(key)

bateriaDisponivel = 100

my_awsmqtt_client.connect()
class Paciente:
    def __init__(self, nome, idade, genero):
        self.nome = nome
        self.idade = idade
        self.genero = genero


def geraQrCode():
            uri = pyotp.totp.TOTP(key).provisioning_uri(name="italaoPikudo",issuer_name="ItaloJebudo")

            print(uri)

            qrcode.make(uri).save("totp.png")
            
            
def autenticacaoMfa():
            while True:
                user_input = input("Digite o código de verificação do seu Autenticador preferido: ")
                verification_result = totp.verify(user_input)

                if verification_result:
                    print("Verificação bem-sucedida. aguarde 5 segundos enquanto autenticamos...\n")
                    break
                    
                else:
                    print("Verificação falhou. Token incorreto")

def plotagem():
            fig, axs = plot.subplots(2, 1, constrained_layout=True)
            axs[0].plot(FrequenciaCardiacaPaciente)
            axs[0].set_title('Frequencia Cardiaca Marise')
            axs[0].set_xlabel('Numero da Medição')
            axs[0].set_ylabel('Frequencia Cardiaca Paciente')
            
            axs[1].plot(vetEspacoUtilizado, '--',label='Espaço')
            axs[1].plot(vetTempoUtilizado, 'x-', label='Tempo')
            axs[1].set_title('Dados da Maquina')
            axs[1].set_xlabel('Numero da Medição')
            axs[1].set_ylabel('Espaço & Tempo') 
            axs[1].legend()

 
def randomizaFrequenciaCardiaca():
            varRandomica = random.randrange(0,2)
            if(varRandomica == 0):
                    FrequenciaCardiacaPaciente.append(random.randrange(FrequenciaCardiacaPaciente[-1], FrequenciaCardiacaPaciente[-1] + 3))
            elif(varRandomica == 1):
                    FrequenciaCardiacaPaciente.append(random.randrange(FrequenciaCardiacaPaciente[-1] - 2,FrequenciaCardiacaPaciente[-1]))
                    

def decrementa_bateria():
    global bateriaDisponivel
    bateriaDisponivel -= 0.01


def insert_iotCore(nomePaciente,idadePaciente,generoPaciente,frequenciaCardiaca,dataLeitura,espacoUtilizado,tempoUtilizado,zonaDisponibilidade,bateriaDisponivel):
        time.sleep(5)
        topic = "frequenciacardiaca/iotcore"
        
        bateriaDisponivelFormat = "{:.2f}".format(bateriaDisponivel)   
           
        dados = {'nomePaciente': nomePaciente, 'idadePaciente': idadePaciente, 'generoPaciente': generoPaciente, 'frequenciaCardiaca': frequenciaCardiaca, 'dataLeitura': dataLeitura,'espacoUtilizado': espacoUtilizado,'tempoUtilizado': tempoUtilizado, 'zonaDisponibilidade': zonaDisponibilidade, 'bateriaDisponivel': bateriaDisponivelFormat}
        
        msg = json.dumps(dados)
        print(msg)
        

        my_awsmqtt_client.publish(topic, msg, 0)
        tamanhoMensagem = (len(str(msg))) / 1024 
        #my_awsmqtt_client.disconnect()
            
        print(dados)
        print(f"Tamanho da Mensagem em KB = {tamanhoMensagem}")
        decrementa_bateria()
        


def insert_db(nomePaciente,idadePaciente,generoPaciente,frequenciaCardiaca,dataLeitura,espacoUtilizado,tempoUtilizado,zonaDisponibilidade):
    try:  
        mydb = mysql.connector.connect(
            user= "admin",
            password= "urubu100",
            host= "rds-underdogs.cfzmhji6igm5.us-east-1.rds.amazonaws.com",
            database= "Grupo3"
        )

        if mydb.is_connected():
            #db_Info = mydb.get_server_info()
            #print("Conectado ao MySQL Server versão ", db_Info)
            time.sleep(1)
            mycursor = mydb.cursor()

            sql_query = "INSERT INTO FrequenciaCardiacaXPerfomance(nomePaciente,idadePaciente,generoPaciente,frequenciaCardiaca,dataLeitura,espacoUtilizado,tempoUtilizado,zonaDisponibilidade) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
            val = [nomePaciente,idadePaciente,generoPaciente,frequenciaCardiaca,dataLeitura,espacoUtilizado,tempoUtilizado,zonaDisponibilidade]
            mycursor.execute(sql_query, val)
            
            mydb.commit()

            print(mycursor.rowcount, f"registro inserido sobre o Paciente {nomePaciente} na AZ {zonaDisponibilidade}")           
            
            
    except mysql.connector.Error as e:
        print("Erro ao conectar com o MySQL", e)
    finally:
        if(mydb.is_connected()):
            mycursor.close()
            mydb.close()
            #print("Conexão com MySQL está fechada\n")


try:
    Paciente1 = Paciente("Italo", 20, "M")
        
    vetEspacoUtilizado = []
    vetTempoUtilizado = []

    FrequenciaCardiacaPaciente = []

    autenticacaoMfa()
    
    while True:
            
            inicio_processamento = time.time()
            dataLeitura = time.strftime("%Y-%m-%d %H:%M:%S",time.localtime())
            if (len(FrequenciaCardiacaPaciente) == 0):
                FrequenciaCardiacaPaciente.append(random.randrange(57, 120))
 
            else:
                randomizaFrequenciaCardiaca()
            
            fim_processamento = time.time()
            duracao = fim_processamento - inicio_processamento
            espaco = sys.getsizeof(FrequenciaCardiacaPaciente) / (1024 * 1024)

            vetEspacoUtilizado.append(espaco)
            vetTempoUtilizado.append(duracao)

            tamanhodaLista = len(vetTempoUtilizado)
          
            insert_db(Paciente1.nome, Paciente1.idade, Paciente1.genero, FrequenciaCardiacaPaciente[-1],dataLeitura, espaco, duracao, "localhost")
            #insert_iotCore(Paciente1.nome, Paciente1.idade, Paciente1.genero, FrequenciaCardiacaPaciente[-1],dataLeitura, espaco, duracao, "localhost",bateriaDisponivel)
            
            #if len(FrequenciaCardiacaPaciente) % 10 == 0:
            #   plotagem()
                  
except KeyboardInterrupt:
    pass

Verificação falhou. Token incorreto
Verificação bem-sucedida. aguarde 5 segundos enquanto autenticamos...

1 registro inserido sobre o Paciente Italo na AZ localhost
1 registro inserido sobre o Paciente Italo na AZ localhost
1 registro inserido sobre o Paciente Italo na AZ localhost


: 

: 